<a href="https://colab.research.google.com/github/nasiksami/Python-Fun-Projects/blob/main/Recurrent_Neural_Network_Play_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#####From Romeo and Juliet play we will feed our model with many data. Then the model will predict the next character based on the previous character. So we will input some starting point for the play and eventually the model will make up the whole lines by combining all the predicted characters


In [1]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [2]:
#data import
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
#we can import out own dataset with.txt format
#from google.colab import files
#path_to_file = list(files.upload().keys())[0]

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [5]:
#now we will proceed with the romeo juliet file 
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [6]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [7]:
#Encoding
#convert it into integer. All the characters.

vocab = sorted(set(text))  #to sort all the unique characters
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}  #mapping
idx2char = np.array(vocab) #making list or array to use the indexes

def text_to_int(text):
  return np.array([char2idx[c] for c in text]) #convert every single characters from the text into their integer representation 

text_as_int = text_to_int(text)

In [8]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [9]:
def int_to_text(ints):
  try:
    ints = ints.numpy() #if its not already a numpy array, it will make it in numpy array
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:133]))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die t


In [10]:
#training
#whatever input we have, we will train in such a way that the output is one character ahead
#for example We will Input Cana the O/P will be anad
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int) #converts string to characters

In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [12]:
#split input target
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [13]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [14]:
#Batch her means 64 different sequences of training examples
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

###Building the Model
Now it is time to build the model. We will use an embedding layer a LSTM and one dense layer that contains a node for each unique character in our training data. The dense layer will give us a probability distribution over all nodes.

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [16]:
#creating a loss function

In [17]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [18]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 6.7274657e-04 -5.0319202e-04 -4.6850252e-04 ... -1.3898171e-03
    1.6326888e-04  1.9410336e-03]
  [ 6.7354497e-03  2.2096853e-03 -4.2898953e-03 ... -1.4658133e-03
    2.4623428e-03  1.5205378e-04]
  [ 4.2199073e-03  1.4884276e-03 -2.6576980e-03 ...  6.0320632e-03
   -2.5269757e-03  4.2872038e-03]
  ...
  [-1.4479773e-04 -1.7485372e-03  4.5530051e-03 ...  8.4786564e-03
   -1.1058331e-03 -1.2404581e-03]
  [-1.0719297e-03 -4.7317124e-04  3.3642091e-03 ...  4.4608871e-03
    4.3269736e-04  1.2044776e-03]
  [-5.6531129e-04  2.7402239e-03  5.4865382e-03 ... -2.0118044e-03
   -2.9888859e-03  1.5888268e-03]]

 [[ 6.7274657e-04 -5.0319202e-04 -4.6850252e-04 ... -1.3898171e-03
    1.6326888e-04  1.9410336e-03]
  [-4.0302812e-03 -1.2220307e-03 -2.2942881e-04 ... -5.2367169e-03
   -1.9886238e-03  3.7209084e-03]
  [ 4.6400935e-03  2.2028410e-03 -4.1117007e-03 ... -3.6126545e-03
    6.3564366e-04  1.7407113e-03]
  ...
  [-5.9013148e-03  1.7260138e-03  3.3845860e-03 ...  9.8720584e

In [19]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 0.00067275 -0.00050319 -0.0004685  ... -0.00138982  0.00016327
   0.00194103]
 [ 0.00673545  0.00220969 -0.0042899  ... -0.00146581  0.00246234
   0.00015205]
 [ 0.00421991  0.00148843 -0.0026577  ...  0.00603206 -0.00252698
   0.0042872 ]
 ...
 [-0.0001448  -0.00174854  0.00455301 ...  0.00847866 -0.00110583
  -0.00124046]
 [-0.00107193 -0.00047317  0.00336421 ...  0.00446089  0.0004327
   0.00120448]
 [-0.00056531  0.00274022  0.00548654 ... -0.0020118  -0.00298889
   0.00158883]], shape=(100, 65), dtype=float32)


In [20]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 6.7274657e-04 -5.0319202e-04 -4.6850252e-04 -9.0439426e-05
  2.2186176e-03 -4.2153057e-03  2.5387893e-03 -1.5957992e-03
 -4.2079929e-03 -5.8606951e-03 -3.5182529e-03  3.4699186e-03
  1.8370716e-03  1.6031422e-03  8.6772554e-03 -1.7934158e-03
  2.3158290e-04  2.9231776e-03  3.7459540e-03  1.4502495e-03
  2.3439331e-03  1.2771345e-03  4.0824446e-03  4.1545639e-03
  1.5291943e-03  2.2731654e-03 -1.7526307e-03 -6.4960530e-04
 -1.5813748e-03 -2.5857263e-03  2.0201434e-04 -2.9485051e-03
  4.3277862e-03 -1.0354127e-03 -4.2676581e-03  5.6235085e-04
  2.7679717e-03  4.8067174e-03 -4.0163812e-03  6.0905153e-03
 -1.7487206e-03  1.3212223e-03  4.7131398e-06 -8.3247357e-04
 -1.4839387e-03  2.6353826e-03  2.3818181e-03  6.0473019e-03
 -2.9647106e-03  3.0515757e-03  5.4815160e-03  4.6843477e-03
 -2.0907405e-03  9.0660609e-04 -3.4957391e-03  4.4471002e-04
  4.2588376e-03  2.8613787e-03 -6.4568496e-03  6.3332240e-04
  1.2470410e-03  4.2957452e-04 -1.3898171e-03  1.6326888e-04
  1.941033

In [21]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

'dvL\nseM&RMqXRj:$oPVWA J;JmolRO.kZ-wQC?IyQGZZALzLenpnExmOe:Xg.TcPNpfzvxUTP.jmUVgrwjr33HFYLpZOlHGEtOAR'

In [22]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [23]:
#completing the model
model.compile(optimizer='adam', loss=loss)

In [24]:
#creating checkpoints
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [25]:
#Training
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 12s 67ms/step - loss: 2.5974
Epoch 2/50
172/172 [==============================] - 12s 68ms/step - loss: 1.8904
Epoch 3/50
172/172 [==============================] - 12s 69ms/step - loss: 1.6414
Epoch 4/50
172/172 [==============================] - 12s 69ms/step - loss: 1.5052
Epoch 5/50
172/172 [==============================] - 12s 70ms/step - loss: 1.4233
Epoch 6/50
172/172 [==============================] - 12s 71ms/step - loss: 1.3658
Epoch 7/50
172/172 [==============================] - 12s 72ms/step - loss: 1.3199
Epoch 8/50
172/172 [==============================] - 12s 72ms/step - loss: 1.2808
Epoch 9/50
172/172 [==============================] - 12s 72ms/step - loss: 1.2437
Epoch 10/50
172/172 [==============================] - 13s 73ms/step - loss: 1.2079
Epoch 11/50
172/172 [==============================] - 13s 73ms/step - loss: 1.1713
Epoch 12/50
172/172 [==============================] - 13s 74ms/step - loss: 1.1357
E

In [30]:
#loading the model
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [31]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [32]:
#We can load any checkpoint we want by specifying the exact file to load

In [ ]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

Generating text

In [34]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [35]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: juliet
julietely pardon lead against the tist: so, for thee thy bed,
And bring in cloudy blood of Montague.
Consider your friends at enough.

ANGELO:
Meantime, nay; the yice and old Derby,
Stand for me! O, the state should kiss on him.

ISABELLA:
Well, passion, house.

FRIAR LAURENA:
And live you lent him go this fashion to death,
I will not be allow'd to make King Richard lew him.

DUKE OF YORK:
Give me my boots. If I pires your worship did beat down;
It shall go warrant him with colours;
And where it feather brave beneft an emb.
I have been canched nine my will were so;
And thus I'll give my stranger eyes, or else
Before the house of Lancaster and his.

FLORIZEL:
Pardon me, Kate, Hermione,
I mind with intere be intent of love tell her my servant Catesby
Where nothing can proceed that toucheth us both
